### Notebook for testing and constructing the retrieval of documents from ifixit
##### Features

- _write_searchphrase_ (arg: search_info, returns: search_phrase): rewriting info (str) into search query
- _load_guides_ (arg: search_phrase, returns: guides): loading guides from ifixit using search phrase, stripping words if no guides are found
- _load_ifixit_guides_ (arg: search_info, returns: guides): both functions combined

In [6]:
#development
from dotenv import load_dotenv
import os
from groq import Groq
import warnings
warnings.filterwarnings("ignore")
#production
from langchain_community.document_loaders import IFixitLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain import hub
from langchain.prompts.prompt import PromptTemplate


In [7]:
#initialize test model
def init_test_model():
    load_dotenv()
    groq_key = os.getenv('GROQ_API_KEY')
    client = Groq(api_key=groq_key)
    return client


In [8]:
#function for rewriting info into searchphrase
def write_searchphrase(search_info: str, debug: bool = False):
    """
    Uses the LLM to rewrite input into a structured searchphrase iFixit searches.

    Args:
        search_info (str): The information to be turned into a searchphrase.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system",
             "content": """Rewrite the following info into a structured searchphrase for iFixit, 
             ensuring it includes in this order the device name, model and part that needs to be repaired. 
             Return only the searchphrase, do not include any other words or comments.
             Capitalize the first letter of each word.
             The searchphrase should be a single sentence."""},
            {"role": "user", "content": search_info},
        ],
        model="llama3-8b-8192",
        temperature=0
    )
    search_phrase = chat_completion.choices[0].message.content
    if debug == True:
        print('Full searchphrase:', search_phrase)
    return search_phrase


#load guides from iFixit
def load_guides(search_phrase: str, debug: bool = False):
    """
    Load a guide from IFixit based on the search phrase.
    If no guide is found, iteratively remove the last word and retry.
    
    Args:
        search_phrase (str): The phrase to search for in IFixit guides.
        
    Returns:
        guides: The loaded guide data or None if no guide is found.
    """
    words = search_phrase.split()
    
    while words:
        query = " ".join(words)
        guides = IFixitLoader.load_suggestions(query, doc_type='guide')
        
        if guides:
            if(debug == True):
                print('Used words:', words)
            return guides  # Return results if found
        
        words.pop()  # Remove the last word and retry

    print('No guides found')
    return None  # Return None if no guide is found


#loading guides from iFixit using optimized searchphrase
def load_ifixit_guides(search_info: str, debug: bool = False):


    search_phrase = write_searchphrase(search_info, debug=debug)
    guides = load_guides(search_phrase, debug=debug)
    return guides



In [9]:
#demonstration of the function
client = init_test_model()
guides = load_ifixit_guides("My washing machine is broken. It's a Bauknecht WMT Pro Eco 6ZB", debug=True) 
print(guides)

Full searchphrase: Bauknecht WMT Pro Eco 6ZB Washing Machine Repair
Used words: ['Bauknecht', 'WMT']
[Document(metadata={'source': 'https://www.ifixit.com/Guide/Bauknecht+WMT+EcoStar+732+Di+F-06+F06+broken+cable/161945', 'title': 'Bauknecht WMT EcoStar 732 Di F-06 F06 broken cable'}, page_content='# Bauknecht WMT EcoStar 732 Di F-06 F06 broken cable\nError F06/ F-06 is displayed when one of the cables controlling the washing machine motor is broken at the connector diagonally below the drum.\n\nSolution: Either reinsert the cable into the connector (impossible without special tools) or "bridge" the connector.\n\n\n###Tools Required:\n\n - Kreuzschlitz PH2\n\n - Sechskant Steckschlüssel 8mm\n\n - T20 Torx Screwdriver Bit\n\n - Wire Stripping Tool\n\n - Flush Cutter\n\n\n###Parts Required:\n\n - Stromkabel (1-adrig)\n\n - Lüsterklemme\n\n\n## Opening the washing machine\nDisconnect the washing machine from the power supply\nRemove the screws from the back of the washing machine as well a